In [4]:
    filename = "LakeTheo.xyz"
    ascii_xyz = []
    externalfile = open(filename,'r') # create connection to file, set to read (r), file must exist
    for line in externalfile:
        if line[0] == '#' or line[0] == "D" : # skip comments and header row
            continue
        else:
            ascii_xyz.append([str(n) for n in line.strip().split()])
    externalfile.close()
    ascii_xyz_len = len(ascii_xyz) # how many rows
    print(ascii_xyz[0]) # print first row

['301493.230623291573', '3814195.86687662033', '0']


In [40]:
nrows = 214
ncols = 344
nlay = nrows*ncols # a dummy dimension
if(nrows*ncols == ascii_xyz_len):
    print('Yay! Sizes match - proceede')

Yay! Sizes match - proceede


In [53]:
ascii_RCZ = [[0 for i in range(ncols)] for j in range(nrows)] # prepopulate RC with zeros (will assign correct values below)
index=0
for irow in range(nrows):
    for jcol in range(ncols): # process as if data are column-wise (probably are)
        ascii_RCZ[irow][jcol] = float(ascii_xyz[index][2]) # put the correct elevation in place
        index = index+1 # increment the counter

# should now have the watershed in grid structure for the DTRM program, 
# not quite done yet as it needs to be written into proper format for FORTRAN program, but well along the way!

In [55]:
# Find the outlet (assume lowest cell bigger than zero)
lowElev = 99999 # start widda big number
for irow in range(nrows):
    for jcol in range(ncols):
        if(ascii_RCZ[irow][jcol] > 0): 
            if(ascii_RCZ[irow][jcol] <= lowElev):
                lowElev = ascii_RCZ[irow][jcol]
                outrow = irow
                outcol = jcol

print(irow,jcol,lowElev) # this is our outlet information

213 343 797.01318359375


In [58]:
# Now build the mask grid to contain the boundary location
# Mask value == 0 if no particle cell
# Mask value = 1 if particle cell
part_mask = [[0 for i in range(ncols)] for j in range(nrows)] # initialize the mask all zeros

for irow in range(nrows):
    for jcol in range(ncols): # process as if data are column-wise (probably are)
         if (ascii_RCZ[irow][jcol] > 0.0):
                part_mask[irow][jcol]=1
                
#print(part_mask[100]) #looks like the right structure

In [60]:
# Now need to write an output file in FORTRAN structure
filename = "LakeTheo.mat.dat"
externalfile = open(filename,"w") # "w" clobbers content already there, consider if "a" is better choice!
# try writing a single row
for irow in range(100,101): # should be a single row
    for jcol in range(ncols): # process as if data are column-wise (probably are)
        message = '  '.join(map(repr, ascii_RCZ[irow][0:ncols])) + "\n"
    externalfile.write(message)
externalfile.close()

In [42]:
cols = 3
rows = 5
arr = [[0 for i in range(cols)] for j in range(rows)]
for i in range(rows):
    print(arr[i])  # print entire row

[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]


In [52]:
myrow = 100
startCol = 0
endCol = 10

print(ascii_RCZ[myrow])

[0.0, 0.0, 0.0, 0.0, 0.0, -99999.0, -99999.0, -99999.0, -99999.0, -99999.0, -99999.0, -99999.0, -99999.0, -99999.0, -99999.0, -99999.0, 979.0, 980.0, 979.0, 977.0, 976.038330078125, 976.03369140625, 976.029052734375, 977.0, 976.015869140625, 976.01123046875, 976.006591796875, 976.0, 976.0, 976.0, 976.0, 975.031005859375, 975.0263671875, 975.0244140625, 975.019775390625, 976.0, 975.006591796875, 975.0, 974.0, 973.07666015625, 973.072021484375, 973.0673828125, 973.062744140625, 973.05810546875, 973.053466796875, 973.048828125, 974.0, 974.0, 973.040283203125, 974.0, 977.0, 978.0, 978.0, 978.0, 976.0, 975.0, 973.0, 970.0, 965.0, 963.004638671875, 964.0, 965.0, 964.0, 961.0, 956.0, 954.0, 956.0, 950.0, 942.0, 938.0, 936.004638671875, 939.0, 942.0, 947.0, 954.0, 956.0, 950.0, 943.0, 942.0, 940.0, 939.0, 937.0, 932.0, 925.0, 919.0, 917.0, 916.006591796875, 916.0, 916.0, 916.0, 921.0, 929.0, 936.0, 936.0, 929.0, 919.0, 920.0, 929.0, 937.0, 948.0, 953.0, 958.0, 960.0, 961.0, 962.0, 961.0, 957.0

In [33]:
ascii_RCZ[0:4]

[0]

In [46]:
ascii_xyz[742:346]

[['310609.881877947366', '3814195.86687662033', '0'],
 ['310636.538752960972', '3814195.86687662033', '0'],
 ['301493.230623291573', '3814169.21000160649', '0'],
 ['301519.887498305179', '3814169.21000160649', '0']]

In [48]:
ascii_RCZ[0][342]

[0.0, 0.0]

In [49]:
ascii_RCZ[1][0]

0.0

In [ ]:
  150.000      0.200      4.000      1.000    CMAN,FDEEP,NMAN,ALIFE 
   29.874      1.000   1440.000   1000.000     29.848     29.901     DX,DT,TMAX,D1,DXT,DYT
 31775     100                NP,PRINT_N
    mask
      21    10    396.000       outlet loc and elev
